<a href="https://colab.research.google.com/github/dmswneunju/DeepLearning_signiture/blob/main/ch05_%EC%9D%B8%EA%B3%B5%EC%8B%A0%EA%B2%BD%EB%A7%9D_%EA%B5%AC%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 문제 정의
## multiclass classifier

## 주요코드


### 1. TensorDataset과 Dataloader
*   입력데이터를 쉽게 처리하고, 배치 단위로 잘라서 학습할 수 있게 도와주는 모듈
*   (Tensor)Dataset (class) : 학습시 사용하는 데이터의 feature(x)와 target(y)의 pair로 이루어짐.
 * 아래에서 코드에서는 tensordataset을 사용하여 dataset인스턴스를 생성했지만, 이미지의 사례와 같이 dataset클래스를 상속받아서 커스텀 인스텀스를 생성하는 형태로 많이 사용.
* Dataloader : 학습 시 각 인스턴스에 쉽게 접근할 수 있도록 순회 가능한 객체(iterable)생성

모델 생성 시 x가 들어가서 y가 나온다.  
pytorch에서는 x와 y를 변수에 하나씩 설정하는 것이 아니라 Dataset이라는 class를 만들어 관리.  
Dataset은 x와 y에 대해서 설정해준 값. 목적에 맞게끔 분할 해준 후 초기화 진행.  
DataLoader는 
* batch로 쪼개지는 것, 쪼갤 때 값을 shuffle하는 역할.
* batch size만큼 모델에 넣어 학습시키고, loss구하고, 역전파 진행, optimizer를 통해 최적화 진행.

### 2. Device설정
* 인공신경망은 GPU를 사용하는 것이 바람직함.
* pytorch에서는 GPU를 사용하여 학습하도록 명시해줄 필요

### 3. 신경망 생성
* **torch.nn**패키지는 신경망 생성 및 학습 시 설정해야하는 다양한 기능 제공
* **nn.Module**을 상속받아 신경망 정의

### 4. Model compile
* 학습 시 필요한 정보들(loss function, optimizer)을 선언
* loss와 optimizer는 변수로 선언하고, 변수를 train/test시 참고할수 있도록 배개변수로 지정  

learning_rate = 0.01  
loss = nn.CrossEntropyLoss()  
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### 5. Train
* 신경망의 학습과정을 별도의 함수로 구성하는 것이 일반적
 * feed forward -> loss -> error back propagation -> (log)(loss가 얼만큼인지 print로 출력 or log파일로 저장) -> (반복)

### 6. Test
* 학습과정과 비슷하나 error back propagate하는 부분이 없다.
 * feed forward -> loss -> (log) -> (반복)

### 7. Iteration
* 신경망 학습은 여러 epochs을 반복해서 수행하면서 모델을 구성하는 최적의 파라미터를 찾는다.
* 지정한 epochs 수만큼 학습과 평가를 반복하면서 모델의 성능(loss, accuracy 등)을 체크

# Basic Neural Network
iris데이터셋을 사용하여 꽃의 품종 구분하는 분류기를 신경망을 사용하여 구현

## [Step1] Load libraries & Datasets

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

# 데이터 불러오기
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['label'] = iris.target

# 데이터 분할
y = df['label']
X = df.drop(['label'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X.values, y.values, random_state=42, stratify=y) 
# stratify:각각의 품종이 동일한 비율로 train, test에 들어가도록

## [Step2] Create DataLoader


In [ ]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
y_test = torch.tensor(y_test, dtype=torch.int64)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_dataloader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=10, shuffle=True)

## [Step3] Set Network Structure

In [ ]:
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    self.input_layer = nn.Linear(4,16)
    self.hidden_layer1 = nn.Linear(16, 32)
    self.output_layer = nn.Linear(32,3)
    self.relu = nn.ReLU()

  def forward(self, x):
    out = self.relu(self.input_layer(x))
    out = self.relu(self.hidden_layer1(out))
    out = self.output_layer(out)
    return out

## [Step4] Create Model instance

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'device = {device}')

model = NeuralNetwork().to(device)

device = cpu


## [Step5] Model compile

In [ ]:
# 모델 컴파일
learning_rate = 0.001
loss = nn.CrossEntropyLoss() #다중 분류이므로 cross entropy 사용
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

## [Step6] Set train loop

In [ ]:
def train_loop(train_loader, model, loss_fn, optimizer):
  size = len(train_loader.dataset)

  for batch, (X,y) in enumerate(train_loader):
    X, y = X.to(device), y.to(device)
    pred = model(X)

    #손실 계산
    loss = loss_fn(pred, y)

    #역전파
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    loss, current = loss.item(), batch*len(X)
    print(f'loss: {loss:>7f} [{current:>5d}]/{size:5d}')

## [Step7] Set test loop

In [ ]:
def test_loop(test_loader, model, loss_fn):
  size = len(test_loader.dataset)
  num_batches = len(test_loader)
  test_loss, correct = 0,0

  with torch.no_grad():
    for X, y in test_loader:
      X,y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1)==y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:8f}\n')

## [Step8] Run model

In [ ]:
# 모델 실행
epochs = 10

for i in range(epochs):
  print(f'Epoch {i+1} \n--------------')
  train_loop(train_dataloader, model, loss, optimizer)
  test_loop(test_dataloader, model, loss)
print('Done!')

Epoch 1 
--------------
loss: 1.097464 [    0]/  112
loss: 1.019018 [   10]/  112
loss: 0.962196 [   20]/  112
loss: 1.128915 [   30]/  112
loss: 1.049655 [   40]/  112
loss: 1.170791 [   50]/  112
loss: 1.094822 [   60]/  112
loss: 0.937628 [   70]/  112
loss: 1.073336 [   80]/  112
loss: 0.881836 [   90]/  112
loss: 1.041271 [  100]/  112
loss: 0.950197 [   22]/  112
Test Error: 
 Accuracy: 34.2%, Avg loss: 1.011848

Epoch 2 
--------------
loss: 1.092918 [    0]/  112
loss: 1.066972 [   10]/  112
loss: 0.906169 [   20]/  112
loss: 1.022640 [   30]/  112
loss: 1.146152 [   40]/  112
loss: 1.097861 [   50]/  112
loss: 0.947375 [   60]/  112
loss: 0.936360 [   70]/  112
loss: 0.928108 [   80]/  112
loss: 0.902105 [   90]/  112
loss: 0.952163 [  100]/  112
loss: 0.950624 [   22]/  112
Test Error: 
 Accuracy: 34.2%, Avg loss: 0.978223

Epoch 3 
--------------
loss: 1.076252 [    0]/  112
loss: 0.979155 [   10]/  112
loss: 1.068217 [   20]/  112
loss: 0.887614 [   30]/  112
loss: 0.935192